# Machine Learning

In [10]:
#import necessary libraries 
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


Read Data

In [17]:
#helper functions
def get_title_from_index(index):
	return moviedataset[moviedataset.index == index]["title_duplicate"].values[0]

def get_index_from_title(title):
    if title not in moviedataset['title'].values:
        return -1
    else:
        return moviedataset[moviedataset.title == title]["index"].values[0]



moviedataset = pd.read_csv("cleaned-movie-dataset-index.csv")
moviedataset['title_duplicate'] = moviedataset.loc[:, 'title']
moviedataset['title'] = moviedataset['title'].str.upper()
moviedataset['title'] = moviedataset['title'].str.replace(" ", "")
moviedataset['title'] = moviedataset['title'].str.replace(":", "")
moviedataset.head()


,Unnamed: 0,index,id,title,adult,popularity,budget,revenue,vote_count,vote_average,release_date,release_year,original_language,genre,casts,keywords,recommendations,similar_movies,title_duplicate
0,0,0,19995,AVATAR,False,432.199,237000000,2920357254,28759,7.569,2009-12-15,2009,en,"['Action', 'Adventure', 'Fantasy', 'Science Fi...","['Sam Worthington', 'Zoe Saldaña', 'Sigourney ...","['culture clash', 'future', 'space war', 'spac...","['Capturing Avatar', 'Avatar: Creating the Wor...","['The Reckless Hour', 'MicroPlanet 3D', 'If I ...",Avatar
1,1,1,299534,AVENGERSENDGAME,False,152.323,356000000,2799439100,22908,8.267,2019-04-24,2019,en,"['Adventure', 'Science Fiction', 'Action']","['Robert Downey Jr.', 'Chris Evans', 'Mark Ruf...","['space travel', 'time travel', 'time machine'...","['Avengers: Infinity War', 'Captain Marvel', '...","['Blankman', 'Santos', 'The Masters of Time', ...",Avengers: Endgame
2,2,2,76600,AVATARTHEWAYOFWATER,False,10255.685,460000000,2309660236,6285,7.740,2022-12-14,2022,en,"['Science Fiction', 'Adventure', 'Action']","['Sam Worthington', 'Zoe Saldaña', 'Sigourney ...","['loss of loved one', 'dying and death', 'alie...","['Capturing Avatar', 'Avatar: Creating the Wor...","['Cyber Ninja', 'Spenser: A Savage Place', 'Br...",Avatar: The Way of Water
3,3,3,597,TITANIC,False,137.265,200000000,2187463944,22650,7.892,1997-11-18,1997,en,"['Drama', 'Romance']","['Leonardo DiCaprio', 'Kate Winslet', 'Billy Z...","['drowning', 'evacuation', 'shipwreck', 'icebe...","['The Lion King', 'Pirates of the Caribbean: T...","['The Reckless Hour', 'The Naked Flame', 'Bell...",Titanic
4,4,4,140607,STARWARSTHEFORCEAWAKENS,False,62.371,245000000,2068223624,17860,7.302,2015-12-15,2015,en,"['Adventure', 'Action', 'Science Fiction', 'Fa...","['Harrison Ford', 'Mark Hamill', 'Carrie Fishe...","['android', 'spacecraft', 'space opera']","['Star Wars: The Last Jedi', 'Star Wars: Episo...","['MicroPlanet 3D', 'Geography of the Universe'...",Star Wars: The Force Awakens


Getting the count matrix

In [12]:
features = ['keywords','casts','genre']
intfeatures = ['vote_average']
for feature in features:
    moviedataset[feature] = moviedataset[feature].fillna('')

cv = CountVectorizer()

count_matrix = {}
for feature in features:
    count_matrix[feature] = cv.fit_transform(moviedataset[feature])
simval = {}
for feature in features:
    simval[feature] = cosine_similarity(count_matrix[feature])
     
simval['final'] = simval['keywords'] + simval['genre'] + simval['casts'] 
similarity_matrix = pd.concat([pd.DataFrame(simval['final']), moviedataset['vote_average']], axis=1)
print(similarity_matrix)




             0         1         2         3         4         5         6  \
0     3.000000  1.210453  1.420843  0.223338  1.450585  1.179943  0.929492   
1     1.210453  3.000000  1.223547  0.204666  1.221372  1.962232  1.461533   
2     1.420843  1.223547  3.000000  0.087612  1.011416  1.089062  1.214578   
3     0.223338  0.204666  0.087612  3.000000  0.280598  0.192972  0.080099   
4     1.450585  1.221372  1.011416  0.280598  3.000000  1.220467  0.996715   
...        ...       ...       ...       ...       ...       ...       ...   
7481  0.079207  0.046881  0.043806  1.279858  0.073841  0.047836  0.028270   
7482  0.017234  0.162340  0.131656  0.074103  0.043091  0.026075  0.083644   
7483  0.052414  0.041267  0.043193  0.014086  0.010921  0.000000  0.042790   
7484  0.082874  0.008771  0.000000  0.044544  0.041443  0.000000  0.000000   
7485  0.019139  0.121529  0.031544  0.655926  0.103685  0.101354  0.061070   

             7         8         9  ...      7477      7478    

Time to recommend a movie

In [23]:
#get a user input on a movie

fav_movie = input("Enter Your Favourite Move")

fav_movie = fav_movie.upper()
fav_movie = fav_movie.replace(" " , "")
fav_movie = fav_movie.replace(":" , "")

modern = input("Do you want a recent movie? Y/N")
if modern not in ['Y', 'N']:
    modern = input("Please enter Y or N")


#get index of movie

movieindex = get_index_from_title(fav_movie)
#remove favourite movie from dataset (so wont recc same movie)
while (movieindex == -1):
    print("Movie not in database please select another movie")
    fav_movie = input("Enter Your Favourite Move")
    fav_movie = fav_movie.upper()
    fav_movie = fav_movie.replace(" " , "")
    fav_movie = fav_movie.replace(":" , "")
    movieindex = get_index_from_title(fav_movie)
similar_movies = {}

similar_movies['final'] =  list(enumerate(simval['final'][movieindex]))

sorted_similar_movies = {}

sorted_similar_movies['final'] = sorted(similar_movies['final'],key=lambda x:x[1],reverse=True)


## set threshhold for a 'good' movie
threshhold = 7.5 ##cutoff for a good movie
## Step 8: Print titles of first 10 movies based off of
i=0
for element in sorted_similar_movies['final']:
    # Get the index and rating of the movie
    movie_index = element[0]
    movie_rating = moviedataset.iloc[movie_index]['vote_average']
    movie_year = moviedataset.iloc[movie_index]['release_year']
    if movie_index == movieindex:
            print('Selected Movie:',get_title_from_index(movie_index))
            print('Reccomended Movies:')
    # Check if the rating is above the threshold
    elif movie_rating >= threshhold:
        if modern == 'Y':
            if(movie_year >= 2005):
                print(i+1, ".", get_title_from_index(movie_index))
                i += 1
        else:
            print(i+1, ".", get_title_from_index(movie_index))
            i += 1  
        # Check if we have printed 10 movies
    if i >= 10:
        break

Selected Movie: 3 Idiots
Reccomended Movies:
1 . A Man Called Otto
2 . A Man Called Ove
3 . The Banshees of Inisherin
4 . Chhichhore
5 . Green Book
6 . Bajrangi Bhaijaan
7 . The Intouchables
8 . The Grand Budapest Hotel
9 . Little Miss Sunshine
10 . Instructions Not Included
